In [2]:
import datetime

import numpy as np
import pandas as pd
import yaml
from sqlalchemy import create_engine


# database connections 

In [3]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_co = config['CO_SA']
    config_etl = config['ETL_PRO']

# Construct the database URL
url_co = (f"{config_co['drivername']}://{config_co['user']}:{config_co['password']}@{config_co['host']}:"
          f"{config_co['port']}/{config_co['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
           f"{config_etl['port']}/{config_etl['dbname']}")
# Create the SQLAlchemy Engine
co_sa = create_engine(url_co)
etl_conn = create_engine(url_etl)

# Extract

In [4]:
df_citas = pd.read_sql_table('citas_generales', co_sa)
df_urgencias = pd.read_sql_table('urgencias', co_sa)
df_hosp = pd.read_sql_table('hospitalizaciones', co_sa)

In [5]:
df_hosp.info()

# Transformations

In [6]:
from datetime import  timedelta

df_hosp.rename(columns={'codigo_hospitalizacion':'codigo_servicio'} ,inplace=True)
df_urgencias.rename(columns={'codigo_urgencia':'codigo_servicio'}, inplace=True)
df_citas.rename(columns={'codigo_cita':'codigo_servicio'},inplace=True)

df_citas['tipo_servicio'] = 'citas'
df_urgencias['tipo_servicio'] = 'urgencias'
df_hosp['tipo_servicio'] = 'hospitalizacion'

columns = ['codigo_servicio', 'id_usuario','id_medico','fecha_solicitud','fecha_atencion','hora_atencion','hora_solicitud','tipo_servicio']
trans_servicio = pd.concat([df_hosp,df_urgencias,df_citas],axis=0)
trans_servicio.head()
columns = set(trans_servicio.columns) - set(columns)

trans_servicio.drop(columns= columns,inplace=True)

trans_servicio['fecha_atencion'] = pd.to_datetime(trans_servicio['fecha_atencion'])
trans_servicio['fecha_solicitud'] = pd.to_datetime(trans_servicio['fecha_solicitud'])
trans_servicio['hora_atencion'] = trans_servicio['hora_atencion'].apply(lambda x : timedelta(hours=x.hour, minutes=x.minute,seconds=x.second))
trans_servicio['hora_solicitud'] = trans_servicio['hora_solicitud'].apply(lambda x : timedelta(hours=x.hour, minutes=x.minute,seconds=x.second))
trans_servicio['fecha_hora_atencion'] = trans_servicio['fecha_atencion'] + trans_servicio['hora_atencion']
trans_servicio['fecha_hora_solicitud'] = trans_servicio['fecha_solicitud'] +trans_servicio['hora_solicitud'] 

trans_servicio.reset_index(drop=True,inplace=True)
trans_servicio.head()


In [7]:
from datetime import date

trans_servicio['saved'] = date.today()

# load

In [8]:
trans_servicio.to_sql('trans_servicio',etl_conn,if_exists='replace',index_label='key_trans_servicio')

In [9]:
trans_servicio.info(
    
)

In [10]:
trans_servicio